# Fuzzy Matching

# Module Code

In [9]:
import warnings
import numpy as np
import pandas as pd
import string
import regex as re
import py_stringsimjoin as ssj
import py_stringmatching as sm
from fuzzywuzzy import fuzz
import jellyfish
from tqdm import tqdm
from cleanco import basename
warnings.filterwarnings('ignore')

org_throw = pd.read_excel('D:\\Fuzzy Matching\\throw out words.xlsx')
# org_throw.head()
org_throw['Organizations'] = org_throw['Organizations'].apply(lambda x: (x.split('(')[0])[:-1])
global org_throw

# this is the file paths that read in both Orbis data and external data
ORBIS_INPUT_PATH = 'D:\\Fuzzy Matching\\Nigeria Firm Names - V2.csv'
ORG_INPUT_PATH = 'D:\\Fuzzy Matching\\Subset for org\\org_s2 - V2.csv'

df_orbis = pd.read_csv(ORBIS_INPUT_PATH)
df_org = pd.read_csv(ORG_INPUT_PATH)

def remove_parenthesis(name):
    return re.sub(r'\(.*\)', '', name)


def remove_punctuation(name):
    return name.translate(str.maketrans('', '', string.punctuation))


def remove_double_space(name):
    name = ' '.join(name.split())
    return name


class OrgMatching:
    '''
    NUM_ROW_ORBIS: the number of rows in Orbis that are used for matching purpose
    NUM_ROW_ORG_START: starting matching row of the external data
    NUM_ROW_ORG_END: ending matching row of the external data
    df_org: external data
    df_orbis: Orbis dataset
    '''
    def __init__(self, NUM_ROW_ORBIS, NUM_ROW_ORG_START, NUM_ROW_ORG_END, df_org, df_orbis):
        
        self.NUM_ROW_ORBIS = NUM_ROW_ORBIS
        self.NUM_ROW_ORG_START = NUM_ROW_ORG_START
        self.NUM_ROW_ORG_END = NUM_ROW_ORG_END
        self.df_org = df_org
        self.df_orbis = df_orbis
        self.outdata_orbis = pd.DataFrame()
        self.outdata_org = pd.DataFrame()
        
    
    def OrbisProcessing(self):
        
        self.df_orbis = self.df_orbis[:self.NUM_ROW_ORBIS]
        # data preprocessing: delete the NA value
        self.df_orbis = self.df_orbis[['name_internat']].dropna()

        self.df_orbis['name_original'] = self.df_orbis['name_internat']
        self.df_orbis['name'] = pd.DataFrame(self.df_orbis['name_internat'].apply(str.lower))
        self.outdata_orbis = self.df_orbis[['name_original', 'name']]
        
        self.outdata_orbis.sort_values(by='name', inplace=True)
        
        self.outdata_orbis['name_clean'] = self.outdata_orbis['name'].apply(str.strip) # remove extra spaces
        self.outdata_orbis['name_clean'] = self.outdata_orbis['name_clean'].apply(remove_parenthesis)
        self.outdata_orbis['name_clean'] = self.outdata_orbis['name_clean'].apply(remove_punctuation)
        # Remove something like LNC, PLC
        self.outdata_orbis['name_clean'] = self.outdata_orbis['name_clean'].apply(basename)
        self.outdata_orbis['name_clean'] = self.outdata_orbis['name_clean'].apply(basename)
        self.outdata_orbis['name_clean'] = self.outdata_orbis['name_clean'].apply(remove_double_space)
        self.outdata_orbis = self.outdata_orbis.reset_index(drop = True)
        
        return(self.outdata_orbis)
    
    
    def OrgProcessing(self):
        
        self.df_org = self.df_org[['organizations']].dropna()
        orgs_unextracted = []

        for index, row in self.df_org.iterrows():
            # row is a single-item list with a string surrounded
            # by curly braces. Extract the single item and remove
            # the surrounding curly braces.
            orgs_unextracted.append(row[0][1:-1])
            
        orgs_extracted = []
        # The rows are json-like formatted strings that contain non-quoted
        # information which includes company names, each of which can be extracted 
        # via regex and be treated as a subrow.
        for row in orgs_unextracted:
            row = row.split('},')
            for subrow in row:
                match = re.findall(r'(?:n=)(.*)(?:,)', subrow)
                orgs_extracted.append(match[0])
                
        orgs_extracted = pd.DataFrame(orgs_extracted)
        self.outdata_org = pd.DataFrame(orgs_extracted.iloc[:,0].value_counts())
        self.outdata_org.rename(columns={0: 'freq_org'}, inplace=True)
        self.outdata_org.reset_index(inplace=True)
        self.outdata_org.rename(columns={0: 'name_org'}, inplace=True)

        self.outdata_org = self.outdata_org[self.NUM_ROW_ORG_START:self.NUM_ROW_ORG_END]
        self.outdata_org['name_original'] = self.outdata_org['index']
        self.outdata_org['name_org'] = pd.DataFrame(self.outdata_org['index'].apply(str.lower))
        self.outdata_org['name_org'] = self.outdata_org['name_org'].apply(str.strip)
        self.outdata_org['name_org'] = self.outdata_org['name_org'].apply(remove_parenthesis)
        self.outdata_org['name_org'] = self.outdata_org['name_org'].apply(remove_punctuation)
        self.outdata_org['name_org'] = self.outdata_org['name_org'].apply(basename)
        self.outdata_org['name_org'] = self.outdata_org['name_org'].apply(basename)
        self.outdata_org['name_org'] = self.outdata_org['name_org'].apply(remove_double_space)
        self.outdata_org.drop(columns='index', inplace = True)
        
        return(self.outdata_org)
    
    
def MatchingWord(outdata_org, outdata_orbis):

    list_word = []

    for item in outdata_org['name_org']:
        
        data = pd.DataFrame()
        # use fuzz_ratio as the index of showing how similar two entities are
        data['fuzz_ratio'] = outdata_orbis.apply(lambda x: fuzz.ratio(item, x.name_clean), axis=1)
#         print(data)

        # threshold that will be considered the same string
        if(any(data['fuzz_ratio'] > 95)): # after several testing, 95% is the best threshold, it can be changed based on different circumstances
            list_word.append(item)
    
    Match = pd.DataFrame()

    for i in range(len(list_word)):
        
        data = pd.DataFrame()
        # calculate all the fuzz ratio for each single word
        data['fuzz_ratio'] = outdata_org.apply(lambda x: fuzz.ratio(list_word[i], x.name_org), axis=1)  
        # sort out all the data with a fuzz_ratio over 95%
        df = outdata_org[data['fuzz_ratio'] > 95]
        Match = pd.concat([Match,df], axis=0)

    Match_2 = pd.DataFrame()

    for i in range(len(list_word)):
        data = pd.DataFrame()
        data['fuzz_ratio'] = outdata_orbis.apply(lambda x: fuzz.ratio(list_word[i], x.name_clean), axis=1)     
        df_1 = outdata_orbis[data['fuzz_ratio'] > 95]
        
        Match_2 = pd.concat([Match_2,df_1], axis=0)

    Match = Match.reset_index(drop=True)
    # reorganize the output: put the name of external data with the name in Orbis to see how the algorithm works
    Match.columns = ['Frequency','Name_Organization_Original', 'Name_Organization_Lower']
    Match = Match[Match['Name_Organization_Lower'] != '']
    
    Match_2 = Match_2.reset_index(drop = True)
    Match_2.columns = ['Name_Nigeria_Original', 'Name_Before_Cleaning', 'Name_Organization_Lower']

    # left join to eliminate invalid values
    Match_Final = pd.merge(Match, Match_2, how = 'left')
    
    Match_Final_1 = Match_Final[['Name_Organization_Original', 'Name_Nigeria_Original']]
    Match_Final_1.drop_duplicates(inplace = True)
    Match_Final_1  = Match_Final_1.reset_index(drop = True)

    return(Match_Final_1)

# throw out the international organizations
def ThrowWord(df):
    
    Org_Match = df['Name_Nigeria_Original'].unique()
    list_org_remain = []
    list_throw = org_throw['Organizations'].values

    for org in Org_Match:
        if(org not in list_throw):
            list_org_remain.append(org)
    result = pd.DataFrame(list_org_remain)
    result.columns = ['Remained Words']
    
    return(result)

In [10]:
# A = OrgMatching(384460, 0, 150000, df_org, df_orbis)
A = OrgMatching(25065, 0, 500, df_org, df_orbis)

In [11]:
outdata_orbis = A.OrbisProcessing()

In [12]:
outdata_org = A.OrgProcessing()

In [13]:
Match_Result = MatchingWord(outdata_org, outdata_orbis)

In [14]:
After_Throw = ThrowWord(Match_Result)

In [16]:
Match_Result.dropna()

,Name_Organization_Original,Name_Nigeria_Original
0,Peoples Democratic Party,Peoples Democratic Party
2,Sterling Bank,Sterling Bank Plc
3,Sterling Bank Plc,Sterling Bank Plc
4,Zenith Bank,Zenith Bank Plc
5,Access Bank,Access Bank PLC
6,Nigerian National Petroleum Corporation,Nigerian National Petroleum Corporation (NNPC)
7,Dangote Industries,Dangote Industries (Ethiopia) PLC
8,Dangote Industries,Dangote Industries Limited
9,Dangote Industries Ltd,Dangote Industries (Ethiopia) PLC
10,Dangote Industries Ltd,Dangote Industries Limited
